# Improved interpretability for computer-aided severity assesment of Rethinopathy of Prematurity 
paper to appear as oral presentation at SPIE2019 Medical Imaging, Computer-aided diagnosis track 

In [1]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import h5py
import sys
sys.path.append('./iMIMIC-RCVs/')
sys.path.append('./iMIMIC-RCVs/rcv_utils.py')
sys.path.append('./iMIMIC-RCVs/scripts/keras_vis_rcv/')
sys.path.append('./iMIMIC-RCVs/scripts/keras_vis_rcv/vis/')

import rcv_utils
import keras
import keras.backend as K
import sklearn.model_selection
import sklearn.linear_model
from rop_utils import *
PWD=''

/home/mara/venv/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 1. Loading concept measures from handcrafted features, for each image in the dataset

In [2]:
feat = pd.read_csv('./sample_5kFeatures.csv')
feature_names = feat.columns
feat['Image Name'] = feat['Image Name'].astype(str).str[:-4]
feat=feat.set_index('Image Name')
concepts = ['cti median', 'cti mean', 'curvature median', 'curvature mean', 'Avg segment diameter median', 'Avg point diameter mean']

In [3]:
meas = feat.to_dict('index')
# checking that the features have been successfully loaded
concept_measure = 'cti median'
image_file = 'BEAU-0071_000387_2_Posterior_os_No'
print  '{} measure for input file {} = '.format(concept_measure, image_file), meas[image_file][concept_measure]

cti median measure for input file BEAU-0071_000387_2_Posterior_os_No =  1.023713797


### 2. Loading ROP data

In [7]:
# Note: we cannot share the original images. 
# For more information you can contact me at: 
# mara.graziani@hevs.ch

h5file = '../split0.data/train.h5'
images, labels, original_files, classes = load_rop_data(h5file)
test_images, test_labels, test_original_files, test_classes = load_rop_data('../split0.data/test.h5')
inputs=np.array([swap(sample) for sample in images[:]])
print inputs.shape
test_inputs = np.array([swap(sample) for sample in test_images[:]])
print test_inputs.shape

data
labels
original_files
data
labels
original_files
(3264, 224, 224, 3)
(988, 224, 224, 3)


### 3. Loading the model

In [9]:
# Note: we cannot share the model weights. 
# For more information you can contact me at: 
# mara.graziani@hevs.ch

model = keras.models.load_model('../split0/final_model.h5')
model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[50176,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: dense_1_1/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=1125104, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_1_1/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op u'dense_1_1/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/mara/venv/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.

In [ ]:
model.load_weights('../split0/best_weights.h5')

### 4. Computing the RCVs
For more information about the method you can read the paper
on regression concept vectors for bidirectional explanations
(https://github.com/medgift/iMIMIC-RCVs).

In [ ]:
layers = ['Conv2d_2b_1x1_conv', 
          'Conv2d_2c_3x3_conv', 
          'Mixed_3b_Concatenated', 
          'Mixed_3c_Concatenated',
          'Mixed_4b_Concatenated',
          'Mixed_4c_Concatenated',
          'Mixed_5c_Concatenated'] #'Mixed_5b_Branch_3_b_1x1_conv', , 'Mixed_5c_Branch_3_b_1x1_conv'
max_rep = 3
batch_size=32
for repetition in range(0, 1):
    for l in layers[:]:
        feats=[]
        i=0
        if os.path.isfile('./rcv/phis/'+str(repetition)+'_concepts_phis_'+l+'.npy'):
            print './rcv/phis/'+str(repetition)+'_concepts_phis_'+l+' already esists'
            continue
        get_layer_output = K.function([model.layers[0].input],
                                  [model.get_layer(l).output])
        while i+batch_size<len(inputs):
            feats.append(get_layer_output([inputs[i:i+batch_size]]))
            i+=batch_size
        feats.append(get_layer_output([inputs[i:]]))
        feats=np.array(feats)
        feats=feats.reshape((feats.shape[0]*feats.shape[1]*feats.shape[2], feats.shape[3], feats.shape[4], feats.shape[5]))
        np.save('./rcv/phis/'+str(repetition)+'_concepts_phis_'+l, np.asarray(feats))

In [ ]:
l = layer = layers[-2]
repetition=0
rop_class = 'Plus'
rcvs = {}
acts = np.load('./rcv/phis/'+str(repetition)+'_concepts_phis_'+l+'.npy')
l=layer = layers[-1]
repetition=0
print layers
rop_class = 'Plus'
rcvs = {}
print 'Extracting RCV at layer: ', layer
acts = np.load('./rcv/phis/'+str(repetition)+'_concepts_phis_'+l+'.npy')
for c in concepts[:1]:
    if c not in rcvs.keys():
        rcvs[c]={}
    print 'Analysing concept: ', c
    fvec=[]
    to_keep=[]
    classes=[]
    fvec, to_keep = get_concept_measures_vector(meas, original_files, c)
    print acts.shape
    X=(np.asarray([acts[i].ravel() for i in to_keep], dtype=np.float64))  
    print X.shape
    classes = [get_sample_class(labels, i) for i in to_keep]
    idxs = get_class_indices(classes, rop_class)
    print idxs
    print X[idxs].shape
    reg_score, cv = solve_regression(
                np.squeeze(X[idxs], axis=1), 
                np.squeeze(fvec[idxs], axis=1), 
                random_state=repetition
    )
    print 'Regression output: ', reg_score, ' (', rop_class, ')'
    rcvs[c][rop_class] = cv

In [ ]:
regression_outputs = compute_regression(max_rep, layers[2:], meas, original_files, acts, labels, concepts)

In [ ]:
np.save('preliminary_regression_outputs_def', regression_outputs)

In [ ]:
plot_dynamics(regression_outputs, 'Normal', concepts, layers)

In [ ]:
plot_dynamics(regression_outputs, 'Plus', concepts, layers)